<a href="https://colab.research.google.com/github/sayam47/Mellowmax-RL/blob/master/CartPole_Mellow_Tensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

import warnings
warnings.filterwarnings('error')


EPISODES = 5000

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = list()
        self.gamma = 0.95    # discount rate
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        act_values = self.model.predict(state)[0]
        prob=0
        try:
            prob = np.exp(act_values)/np.sum(np.exp(act_values))
        except:
            print(act_values)
        return np.random.choice(len(prob) , p = prob)

    def replay(self, batch_size):
        minibatch = [self.memory[-1]]
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


env = gym.make('CartPole-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
done = False
batch_size = 1
all_reward = deque(maxlen=5)
for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):
        # env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            all_reward.append(time+1)
            print("episode: {}/{}, score: {}, avg: {}".format(e, EPISODES, time+1, np.mean(all_reward)))
            break
            
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
episode: 0/5000, score: 17, avg: 17.0
episode: 1/5000, score: 28, avg: 22.5
episode: 2/5000, score: 13, avg: 19.333333333333332
episode: 3/5000, score: 43, avg: 25.25
episode: 4/5000, score: 40, avg: 28.2
episode: 5/5000, score: 43, avg: 33.4
episode: 6/5000, score: 15, avg: 30.8
episode: 7/5000, score: 13, avg: 30.8
episode: 8/5000, score: 12, avg: 24.6
episode: 9/5000, score: 10, avg: 18.6
episode: 10/5000, score: 8, avg: 11.6
episode: 11/5000, score: 11, avg: 10.8
episode: 12/5000, score: 10, avg: 10.2
episode: 13/5000, score: 8, avg: 9.4
episode: 14/5000, score: 9, avg: 9.2
episode: 15/5000, score: 10, avg: 9.6
episode: 16/5000, score: 10, avg: 9.4
[33.635326 93.85144 ]


TypeError: ignored